# Preamble

In [1]:
from sympy import symbols, integrate, Piecewise, sympify, simplify, nsimplify, factorial
t, l, b4b5 = symbols("t, l, B₄B₅")

# Introduction

The splines used for corner blending in Prunt are based on those presented by Jiang et al. (2022). Only the working relevant to Prunt is detailed here.

# Analytical Distance Function

To compute the distance along the curve with a given T value we start with Eq. (11) from Jiang et al. (2022). Note that l1 and l2 are both replaced by l as our code uses only symmetrical curves:

In [2]:
u0u7v0v7 = ((2145 * abs(b4b5))**2 - 22500 * l * l - (1995 * l)**2) / (39900 * l)

sigma = [
    15 * l,
    15 * l,
    15 * l,
    15 * l,
    sympify("10/143") * u0u7v0v7 + sympify("133/143") * 15 * l,
    sympify("38/143") * u0u7v0v7 + sympify("105/143") * 15 * l,
    sympify("254/429") * u0u7v0v7 + sympify("175/429") * 15 * l,
    u0u7v0v7,
    sympify("254/429") * u0u7v0v7 + sympify("175/429") * 15 * l,
    sympify("38/143") * u0u7v0v7 + sympify("105/143") * 15 * l,
    sympify("10/143") * u0u7v0v7 + sympify("133/143") * 15 * l,
    15 * l,
    15 * l,
    15 * l,
    15 * l
]

We then multiply these by the bezier basis functions as described by Jiang et al. (2022) to compute the parametric speed function:

In [3]:
def bezier_basis(m, i, t):
    return (factorial(m) / (factorial(i) * factorial(m - i))) * t**i * (1 - t)**(m - i)
bezier_basis(sympify("m"), sympify("i"), sympify("t"))

t**i*(1 - t)**(-i + m)*factorial(m)/(factorial(i)*factorial(-i + m))

In [4]:
speed = sum((sigma[i] * bezier_basis(14, i, t) for i in range(0, 15)))
simplify(speed)

3*(8179600*l**2*t**14 - 57257200*l**2*t**13 + 168908740*l**2*t**12 - 269108840*l**2*t**11 + 244488244*l**2*t**10 - 120240120*l**2*t**9 + 25050025*l**2*t**8 + 408980*l**2*t**7 - 1431430*l**2*t**6 + 1717716*l**2*t**5 - 715715*l**2*t**4 + 1330*l**2 - 8179600*t**14*Abs(B₄B₅)**2 + 57257200*t**13*Abs(B₄B₅)**2 - 168908740*t**12*Abs(B₄B₅)**2 + 269108840*t**11*Abs(B₄B₅)**2 - 244488244*t**10*Abs(B₄B₅)**2 + 120240120*t**9*Abs(B₄B₅)**2 - 25050025*t**8*Abs(B₄B₅)**2 - 408980*t**7*Abs(B₄B₅)**2 + 1431430*t**6*Abs(B₄B₅)**2 - 1717716*t**5*Abs(B₄B₅)**2 + 715715*t**4*Abs(B₄B₅)**2)/(266*l)

Finally we integrate the speed function to compute the distance function:

In [5]:
simplify(integrate(speed, t))

t*(23940*l**2 + 9815520*t**14*(l**2 - Abs(B₄B₅)**2) + 73616400*t**13*(-l**2 + Abs(B₄B₅)**2) + 233873640*t**12*(l**2 - Abs(B₄B₅)**2) + 403663260*t**11*(-l**2 + Abs(B₄B₅)**2) + 400071672*t**10*(l**2 - Abs(B₄B₅)**2) + 216432216*t**9*(-l**2 + Abs(B₄B₅)**2) + 50100050*t**8*(l**2 - Abs(B₄B₅)**2) + 920205*t**7*(l**2 - Abs(B₄B₅)**2) + 3680820*t**6*(-l**2 + Abs(B₄B₅)**2) + 5153148*t**5*(l**2 - Abs(B₄B₅)**2) + 2576574*t**4*(-l**2 + Abs(B₄B₅)**2))/(1596*l)

# Bezier Basis Functions
The bezier basis functions may also be used to compute a point on the bezier curve, as is done in Point_At_T_V2 in Motion.PH_Beziers:

In [6]:
print("Bez.Control_Points (0) * (" + str(bezier_basis(15, 0, t)) + ")")
for i in range(1, 16):
    print("+ Scaled_Position_Offset (Bez.Control_Points (" + str(i) + ")) * (" + str(bezier_basis(15, i, t)) + ")")

Bez.Control_Points (0) * ((1 - t)**15)
+ Scaled_Position_Offset (Bez.Control_Points (1)) * (15*t*(1 - t)**14)
+ Scaled_Position_Offset (Bez.Control_Points (2)) * (105*t**2*(1 - t)**13)
+ Scaled_Position_Offset (Bez.Control_Points (3)) * (455*t**3*(1 - t)**12)
+ Scaled_Position_Offset (Bez.Control_Points (4)) * (1365*t**4*(1 - t)**11)
+ Scaled_Position_Offset (Bez.Control_Points (5)) * (3003*t**5*(1 - t)**10)
+ Scaled_Position_Offset (Bez.Control_Points (6)) * (5005*t**6*(1 - t)**9)
+ Scaled_Position_Offset (Bez.Control_Points (7)) * (6435*t**7*(1 - t)**8)
+ Scaled_Position_Offset (Bez.Control_Points (8)) * (6435*t**8*(1 - t)**7)
+ Scaled_Position_Offset (Bez.Control_Points (9)) * (5005*t**9*(1 - t)**6)
+ Scaled_Position_Offset (Bez.Control_Points (10)) * (3003*t**10*(1 - t)**5)
+ Scaled_Position_Offset (Bez.Control_Points (11)) * (1365*t**11*(1 - t)**4)
+ Scaled_Position_Offset (Bez.Control_Points (12)) * (455*t**12*(1 - t)**3)
+ Scaled_Position_Offset (Bez.Control_Points (13)) * (105*

# Curvature
The equation used for curvature in Motion.PH_Beziers was found by trial and error. At some point in the future we will prove that the equation is correct.

# References

Jiang, X., Hu, Y., Huo, G., Su, C., Wang, B., Li, H., ... & Zheng, Z. (2022). Asymmetrical Pythagorean-hodograph spline-based C⁴ continuous local corner smoothing method with jerk-continuous feedrate scheduling along linear toolpath. *The International Journal of Advanced Manufacturing Technology*, 121(9-10), 5731-5754. https://doi.org/10.1007/s00170-022-09463-y